In [1]:
input = os.environ['SPARK_HOME'] + "/data/mllib/sample_svm_data.txt"
mymodelpath = os.getcwd() + "/test/modelpath"
print input

/Users/egorkharakozov/spark-2.0.0-bin-hadoop2.7/data/mllib/sample_svm_data.txt


In [2]:
from pyspark.mllib.classification import SVMWithSGD, SVMModel
from pyspark.mllib.regression import LabeledPoint
import numpy as np

In [3]:
# Load and parse the data
def parsePoint(line):
    values = [float(x) for x in line.split(' ')]
    return LabeledPoint(values[0], values[1:])

data = sc.textFile(input)
parsedData = data.map(parsePoint)

In [4]:

#It looks like you have two versions of numpy installed and pyspark is importing the older one. To confirm this, you can do the following:

import numpy
print numpy.__version__
print numpy.__path__

1.11.2
['/Users/egorkharakozov/anaconda/lib/python2.7/site-packages/numpy']


In [5]:
os.environ['PYSPARK_DRIVER_PYTHON'] = "/Users/egorkharakozov/anaconda/pkgs/"
os.environ["PYSPARK_PYTHON"] = "/Users/egorkharakozov/anaconda/bin/python2.7"

os.environ

{'SPARK_HOME': '/Users/egorkharakozov/spark-2.0.0-bin-hadoop2.7', 'TERM_PROGRAM_VERSION': '361.1', 'LOGNAME': 'egorkharakozov', 'USER': 'egorkharakozov', 'PYSPARK_SUBMIT_ARGS': '"--name" "PySparkShell" "pyspark-shell"', 'HOME': '/Users/egorkharakozov', 'PATH': '/opt/local/bin:/opt/local/sbin:/Users/egorkharakozov/anaconda/bin:/Library/Frameworks/Python.framework/Versions/2.7/bin:/usr/local/bin:/usr/bin:/bin:/usr/sbin:/sbin:/usr/local/git/bin', 'TERM_PROGRAM': 'Apple_Terminal', 'LANG': 'en_US.UTF-8', 'TERM': 'xterm-color', 'Apple_PubSub_Socket_Render': '/private/tmp/com.apple.launchd.VBLJPgdoZs/Render', 'SHLVL': '2', 'XPC_FLAGS': '0x0', 'OLD_PYTHONSTARTUP': '', 'PYSPARK_DRIVER_PYTHON': '/Users/egorkharakozov/anaconda/pkgs/', '_': '/Users/egorkharakozov/anaconda/python.app/Contents/MacOS/python', 'PYTHONHASHSEED': '0', 'TERM_SESSION_ID': '1FEE180C-B354-445F-B62D-20BF8C2A2FC8', 'XPC_SERVICE_NAME': '0', 'JPY_PARENT_PID': '2043', 'PYTHONPATH': '/Users/egorkharakozov/spark-2.0.0-bin-hadoop2.

In [6]:
import os
import sys
import pandas as pd
#os.environ["PYTHONPATH"] = "/Users/egorkharakozov/anaconda/lib/python2.7/site-packages/ipykernel/resources"


In [5]:
# Build the mod
model = SVMWithSGD.train(parsedData, iterations=100)

In [6]:
# Evaluating the model on training data
labelsAndPreds = parsedData.map(lambda p: (p.label, model.predict(p.features)))
trainErr = labelsAndPreds.filter(lambda (v, p): v != p).count() / float(parsedData.count())
print("Training Error = " + str(trainErr))

Training Error = 0.38198757764


In [7]:
# Save and load model
model.save(sc, mymodelpath)
sameModel = SVMModel.load(sc, mymodelpath)

In [8]:
os.listdir(mymodelpath)

['data', 'metadata']